In [58]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
import time
import pickle
import sys
import regex as re
sys.setrecursionlimit(50000)

In [11]:
#Loops through the years, scrapes Box Office Mojo & returns the result with True or Issue with False

all_movies_html = dict()

for year in range(2010,2020,1):
    all_movies_html[year] = scrape_movie_list_opening_weekend(year)
    time.sleep(.5+2*random.random())

Variable                            Type        Data/Info
---------------------------------------------------------
BeautifulSoup                       type        <class 'bs4.BeautifulSoup'>
HTML_table_row_to_dict              function    <function HTML_table_row_<...>o_dict at 0x7fe31c047670>
all_movies_html                     dict        n=10
currency_to_int                     function    <function currency_to_int at 0x7fe31c047700>
np                                  module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pd                                  module      <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
percent_to_float                    function    <function percent_to_float at 0x7fe31c047790>
pickle                              module      <module 'pickle' from '/h<...>lib/python3.8/pickle.py'>
random                              module      <module 'random' from '/h<...>lib/python3.8/random.py'>
read_from_pickle                    function    

In [24]:
all_movies_clean = dict()
all_errors_clean =dict()

for year in range(2010,2020,1):
    
    if all_movies_html[year][0]:

        movies_log = list()
        error_log = list() 
        counter = 1

        for table_row in all_movies_html[year][1]:

            if table_row.find('td'):

                table_row_data = table_row.find_all('td')
                movie_info_clean = HTML_table_row_to_dict(table_row_data)
                
                if type(movie_info_clean) == dict:
                    movies_log.append(movie_info_clean)

                else:
        #             (counter, movie_info_clean[2]['Title'],len(movie_info_clean[2]),list(movie_info_clean[2])[-1])
#                     print(movie_info_clean)
                    movie_info_clean[1]['Counter'] = counter
                    movie_info_clean[1]['Error Message'] = movie_info_clean[0]
                    error_log.append(movie_info_clean[1])
                counter += 1
                
        all_movies_clean[year] = movies_log
        all_errors_clean[year] = error_log


In [4]:
#convert to dataframe & save as csv

# df_list = pd.concat([pd.json_normalize(all_movies_clean[year]) for year in all_movies_clean],ignore_index=True)

# df_list.to_csv('Scraped_Cleaned_MoviesList_2010-19.csv',index=False)

df_movies = pd.read_csv('Scraped_Cleaned_MoviesList_2010-19.csv',parse_dates=[8])
df_movies.head()

,Release Link,Title,Domestic Gross,Max Theatres,Opening Weekend Gross,Domestic to Opening Gross,Opening Weekend Theatres,Release Date,Studio Link,Studio Name
0,https://www.boxofficemojo.com//release/rl15158...,Iron Man 2,312433331,4390,128122480,0.410,4380,May 7 2010,https://pro.imdb.com/company/co0023400/boxoffi...,Paramount Pictures
1,https://www.boxofficemojo.com//release/rl12485...,Harry Potter and the Deathly Hallows: Part 1,295983305,4125,125017372,0.422,4125,Nov 19 2010,https://pro.imdb.com/company/co0002663/boxoffi...,Warner Bros.
2,https://www.boxofficemojo.com//release/rl33932...,Alice in Wonderland,334191110,3739,116101023,0.347,3728,Mar 5 2010,https://pro.imdb.com/company/co0226183/boxoffi...,Walt Disney Studios Motion Pictures
3,https://www.boxofficemojo.com//release/rl13835...,Toy Story 3,415004880,4028,110307189,0.266,4028,Jun 18 2010,https://pro.imdb.com/company/co0226183/boxoffi...,Walt Disney Studios Motion Pictures
4,https://www.boxofficemojo.com//release/rl57771...,Shrek Forever After,238736787,4386,70838207,0.297,4359,May 21 2010,https://pro.imdb.com/company/co0040938/boxoffi...,DreamWorks


In [5]:
df_movies.head()

,Release Link,Title,Domestic Gross,Max Theatres,Opening Weekend Gross,Domestic to Opening Gross,Opening Weekend Theatres,Release Date,Studio Link,Studio Name
0,https://www.boxofficemojo.com//release/rl15158...,Iron Man 2,312433331,4390,128122480,0.410,4380,May 7 2010,https://pro.imdb.com/company/co0023400/boxoffi...,Paramount Pictures
1,https://www.boxofficemojo.com//release/rl12485...,Harry Potter and the Deathly Hallows: Part 1,295983305,4125,125017372,0.422,4125,Nov 19 2010,https://pro.imdb.com/company/co0002663/boxoffi...,Warner Bros.
2,https://www.boxofficemojo.com//release/rl33932...,Alice in Wonderland,334191110,3739,116101023,0.347,3728,Mar 5 2010,https://pro.imdb.com/company/co0226183/boxoffi...,Walt Disney Studios Motion Pictures
3,https://www.boxofficemojo.com//release/rl13835...,Toy Story 3,415004880,4028,110307189,0.266,4028,Jun 18 2010,https://pro.imdb.com/company/co0226183/boxoffi...,Walt Disney Studios Motion Pictures
4,https://www.boxofficemojo.com//release/rl57771...,Shrek Forever After,238736787,4386,70838207,0.297,4359,May 21 2010,https://pro.imdb.com/company/co0040938/boxoffi...,DreamWorks


In [ ]:
# write_to_pickle("Movies_list_HTML.pickle",all_movies_html)

In [4]:
def write_to_pickle(filename, var):
    """
    Write the given variable's contents to a pickle file with provided filename
    """
    
    with open(filename+".pickle",'wb') as to_write:
        pickle.dump(var,to_write)
        
def read_from_pickle(filename):
    """
    Read & return data from the given pickle filename
    """
    
    with open(filename+".pickle",'rb') as to_read:
        var = pickle.load(to_read)
    return var

In [5]:
def scrape_movie_list_opening_weekend(year):
    """
    Scrapes the list of movies present in a table format in Box Office Mojo
    Checks if the required table data was retrieved from the website
    Returns the parsed HTML text for the table portion
    
    Input: Year
    Output: Tuple containing True or False. 
            if True, returns table rows in HTML format containing movie list.
            if False, returns Error message or reason for False
    """
     
    url = "https://www.boxofficemojo.com/year/{}/?sort=openingWeekendGross&grossesOption=totalGrosses"

# Gets data from Box Office Mojo. Checks if status code is not 200. If not 200, returns status code & False
# If status code 200, parses the text & looks for table, table row & movie title in second row, to confirm
# formatting is similar. If not, returns the same, else errors out.

    try:
        response = requests.get(url.format(year))
        if response.status_code != 200:
            return (False, "For year {} Got status code: {}".format(year,response.status_code))
        
        else:
            soup = BeautifulSoup(response.text, 'html5lib')
            
            try:
                movie_title = soup.find('div', id='table').find('tbody').find_all('tr')[1].find_all('td')[1].text
                if movie_title:
                    return (True, soup.find('div', id='table').find('tbody').find_all('tr')[1:])
                else:
                    return (False, "No movie title in second <tr> row. Different HTML formatting")
            except Exception as error_message:
                return (False, "Retrieving table/movie title gave following exception: {}".format(error_message))
            
    except Exception as error_message:
        return (False, "Error Somewhere in entire block with message: {}".format(error_message))

In [53]:
def HTML_table_row_to_dict(table_row_data):
    """
    Takes in table_row_data for a single table row in HTML syntax for Box Office Mojo 
    and returns a dictionary with necessary values 
    """
    
    temp_dict = dict()
    try:

        temp_dict['Release Link'] = "https://www.boxofficemojo.com/"+table_row_data[1].find('a').get('href')
        temp_dict['Title'] = table_row_data[1].text
        temp_dict['Domestic Gross'] = currency_to_int(table_row_data[5].text)
        temp_dict['Max Theatres'] = table_row_data[6].text.strip().replace(',','')
        temp_dict['Opening Weekend Gross'] = currency_to_int(table_row_data[7].text)
        temp_dict['Domestic to Opening Gross'] = percent_to_float(table_row_data[8].text)
        temp_dict['Opening Weekend Theatres'] = table_row_data[9].text.strip().replace(',','')
        temp_dict['Release Date'] = table_row_data[10].text + " "+str(year)
        temp_dict['Studio Link'] = table_row_data[12].find('a').get('href')
        temp_dict['Studio Name'] = table_row_data[12].text.strip()
    except Exception as ex:
        return (ex, temp_dict)

    return temp_dict


def currency_to_int(amount):
    """
    Converts a given curreny string to an integer
    
    Input: amount --> '$123,562,324'
    Output: Output --> 123562324
    """
    try:
        return int(amount.strip('$').replace(',',''))
    except :
        return np.nan

def percent_to_float(percentage):
    """
    Converts a given string percentage to float
    
    Input: 90.4%
    Ouput: 0.904
    """
    
    try:
        return float(percentage.strip('%-'))/100
    except:
        return np.nan

def runtime_to_minutes(runtimestring):
    """
    Take duration in string format & return in minute integer
    
    Input: 2 hrs 53 mins
    Output: 173
    """
    
    runtime = runtimestring.split()
    try:
        return int(int(runtime[0])*60 + int(runtime[2]))
    except:
        return np.nan
    
def get_movie_value(soup, field_name):
    

    
    obj = soup.find(text=re.compile(field_name))
    print(obj)
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [307]:
#NOT SURE IF I WILL NEED THE BELOW FUNCTION as a FUNCTION


def scrape_movies_list(start_year,end_year):
    """
    Scrapes the movies list from box office mojo and returns the relevant info in HTML format
    
    Input: Start Year --> eg: 2010
           End Year --> eg: 2019
    Output: Dictionary with keys for each year & value containing HTML format of all movie table rows"""
    for year in range(start_year,end_year+1,1):
        continue
    return None


In [ ]:
#Initial Testing Code
# Delete in the end

soup = BeautifulSoup(response.text, 'html5lib')
all_tr = soup.find('div', id='table').find('tbody').find_all('tr')
all_tr[1].find_all('td')[1].text

movies_2020 = list()
error_log = list()
counter = 1

for table_row in all_tr:
    
    if table_row.find('td'):
        
        table_row_data = table_row.find_all('td')
        
        movie_info_clean = HTML_table_row_to_dict(table_row_data)
        if type(movie_info_clean) == dict:
            movies_2020.append([movie_info_clean])
            
        else:
#             (counter, movie_info_clean[2]['Title'],len(movie_info_clean[2]),list(movie_info_clean[2])[-1])
            movie_info_clean[1]['Counter'] = counter
            movie_info_clean[1]['Error Message'] = movie_info_clean[0]
            error_log.append([movie_info_clean[1]])
        counter += 1

all_tr[210].find_all('td')

In [6]:
url = "https://www.boxofficemojo.com/release/{}/?ref_=bo_yld_table_1"

release_id = "rl3059975681"

response = requests.get(url.format(title_id))

soup_release = BeautifulSoup(response.text,'html5lib')
soup.prettify()

In [32]:
movie_dict = dict()

In [73]:
title_id = soup_release.find('a', class_='mojo-title-link').get('href').split('/')[2]

summary_table = soup_release.find('div', class_='mojo-summary-values').find_all('div')
print(len(summary_table))

# movie_dict['Release Link'] = release_url
movie_dict['Budget'] = currency_to_int(summary_table[2].find('span', class_='money').text)
# movie_dict['MPAA Rating'] = summary_table[4].find('span')[1].text
# movie_dict['Running Time'] = summary_table[5].find_all('span')[1].text.split()

summary_table2 = soup_release.find('div', class_='mojo-summary-values')
runtime_to_minutes(summary_table2.find(text=re.compile("Running")).findNext().text)
currency_to_int(summary_table2.find(text=re.compile("Budget")).findNext().text)
summary_table2.find(text=re.compile("MPAA")).findNext().text.split()
summary_table2.find(text=re.compile("Genre")).findNext().text.split()

# movie_dict


8


356000000

In [11]:
# https://www.boxofficemojo.com/title/tt4154796/credits/?ref_=bo_tt_tab#tabs
    
url = "https://www.boxofficemojo.com/title/{}/credits/?ref_=bo_tt_tab#tabs"

title_id = "tt4154796"

response = requests.get(url.format(title_id))

soup_release = BeautifulSoup(response.text,'html5lib')
soup.prettify()

'<!DOCTYPE html>\n<html class="a-no-js" data-19ax5a9jf="dingo">\n <head>\n  <script>\n   var aPageStart = (new Date()).getTime();\n  </script>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n  <meta charset="utf-8"/>\n  <title dir="ltr">\n   Avengers: Endgame - Box Office Mojo\n  </title>\n  <meta content="Avengers: Endgame" name="title"/>\n  <meta content="Box Office Mojo" property="og:site_name"/>\n  <meta content="telephone=no" name="format-detection"/>\n  <link href="https://m.media-amazon.com/images/G/01/boxofficemojo/v2/favicon._CB448965889_.ico" rel="icon" type="image/x-icon"/>\n  <link href="https://images-na.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|012LjolmrML.css,51AZ-Jz5kmL.css,51IB+wfP8qL.css,01evdoiemkL.css,01K+Ps1DeEL.css,01Vctty9pOL.css,314djKvMsUL.css,01ZTetsDh7L.css,11cMnOipjJL.css,01pbA9Lg3yL.css,21LK7jaicML.css,11L58Qpo0GL.css,21kyTi1FabL.css,01ruG+gDPFL.css,01YhS3Cs-hL.css,21GwE3cR-yL.css,11KLBtpWIAL.css,1